# Variable definitions

In [54]:
%% General 
sav=1;                                    % to save results
gtpath='SIM/Test/object.tif';             % file name ground truth 
expFolder='DNN4SIM_data/simulated_sim';   % experiment folder

% Image size
sz = [1024,1024];

%% PSF
lamb=488;                % Illumination wavelength
res=32;                  % Resolution (nm)
Na=1.49;                 % Objective numerica aperture
nl=1.518;                % Refractive index of the objective medium (glass/oil)
ns=1.333;                % Refractive index of the sample medium (water)

%% Patterns
orr=[0 pi/3 2*pi/3] + pi/12;   % Patterns orientations (vector)                
ph=linspace(0,pi/4,3); % Patterns lateral phases (vector)
%ph=ph(1:end-1);  
a=0.9;                 % Amplitude coefficient 
bet=asin(Na/nl);       % Angle between side beams and the optic axis (e.g. bet asin(Na/nl))
wf=0;              	   % Boolean true to add a widefield image in the SIM acquisition

%% Acquisition
downFact=[2 2];  % Downsmpling factor (e.g. [2 2 2]) 
photBud=500;    % Photon Budget

% add necessary paths
addpath('SIM/Utils')
run 'SIM/GlobalBioIm-master/setGlobalBioImPath.m'
javaaddpath 'SIM/Utils/PSFGenerator.jar'

# Pattern Generation

In [55]:
%% PSF Generation
fprintf('PSF Generation ...........');
fc=2*Na/lamb*res;
ll=linspace(-0.5,0,sz(1)/2+1);
lr=linspace(0,0.5,sz(1)/2);
[X,Y]=meshgrid([ll,lr(2:end)],[ll,lr(2:end)]);
[th,rho]=cart2pol(X,Y);
OTF=fftshift(1/pi*(2*acos(abs(rho)/fc)-sin(2*acos(abs(rho)/fc))).*(rho<fc));
figure;subplot(1,2,1);imagesc((fftshift(OTF))); axis image; title('OTF'); colormap(fire(200));viscircles(floor(sz/2)+1,fc*sz(1));
psf=real(fftshift(ifft2(OTF)));
subplot(1,2,2);imagesc(psf); axis image; title('PSF'); caxis([0 0.01*max(psf(:))]);
fprintf(' done \n');

%% Patterns Generation
fprintf('Patterns Generation ......');
patt=zeros([sz(1:2),length(orr)*length(ph)]);
[X,Y]=meshgrid(0:sz(2)-1,0:sz(1)-1);X=X*res;Y=Y*res;
it=1;
for ii=1:length(orr)
    k=2*pi*ns/lamb*[cos(orr(ii)), sin(orr(ii))]*sin(bet);
    for jj=1:length(ph)
        patt(:,:,it)=1+ a*cos(2*(k(1)*X+k(2)*Y + ph(jj)));
        it=it+1;
    end
end
if wf, patt(:,:,end+1)=ones(sz(1:2));
end
nbPatt=size(patt,3); % Normalization such that the mean of each pattern is 1/#Patterns
for ii=1:nbPatt
    tmp=patt(:,:,ii);
    patt(:,:,ii)=patt(:,:,ii)/(mean(tmp(:))*nbPatt);
end
figure;subplot(1,2,1);imagesc(patt(:,:,1)); axis image; title('Example pattern');
subplot(1,2,2);imagesc(log(1+abs(fftshift(fftn(patt(:,:,1)))))); axis image; title('Example pattern FFT'); 
viscircles(floor(sz(1:2)/2)+1,fc*sz(1));
fprintf(' done \n');

save([expFolder,'/PSF'],'psf');
save([expFolder,'/OTF'],'OTF');
OTF = imresize(OTF, 0.5);
saveastiff(single(fftshift(OTF)),[expFolder,'/OTF.tif']);
saveastiff(single(psf),[expFolder,'/psf.tif']);
saveastiff(single(patt),[expFolder,'/patterns.tif']);

PSF Generation ........... done 
Patterns Generation ...... done 


In [41]:
imshow(patt(:,:,1))

# Importing the dataset

In [42]:
from nd2reader import ND2Reader
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

%matplotlib widget

import sys  
sys.path.insert(0, 'lib')
from iplabs import IPLabViewer as viewer
# Print colors
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# Load the sim_output image from 'path'
def load_sim_output(path, channels):
    # Read image
    sim_in = ND2Reader(path)
    sim_ch = []
    sim_in.iter_axes = 'c'
    for i, channel in enumerate(sim_in):
        # Extract the required channel
        if sim_in.metadata['channels'][i] in channels:
            # Check the size
            if channel.shape != (1024,1024):
                print(f'{bcolors.FAIL}Size problem with file {path}, skipping this file.{bcolors.ENDC}')
                raise ValueError()

            sim_ch.append(np.array(channel))
    
    return sim_ch
    
# Load the entire sim dataset into a numpy array
def load_sim_dataset(path_to_raw, channels):
    ls = []
    path_to_raw = Path(path_to_raw)

    # Loop through all image folders and add the sim images to the dataset
    print('Collecting images...   0%', end='\r')
    subdir_list = [f for f in path_to_raw.glob('*')]
    for i, f in enumerate(subdir_list):
        # Output images
        output_path = f / 'SIM_output.nd2'
        try:
            # Append images to labels
            ls = ls + load_sim_output(output_path, channels)
        except (FileNotFoundError, ValueError) as err:
            if type(err) == FileNotFoundError:
                print(f'{bcolors.FAIL}File missing: {output_path}{bcolors.ENDC}')
                continue
            elif type(err) == ValueError:
                continue
        print(f'Collecting images... {i/len(subdir_list)*100:3.0f}%', end='\r')
    
    print(f'Collecting images... {bcolors.OKGREEN}100%{bcolors.ENDC}')
        
    return np.array(ls)

# Augment dataset (add a 180 degree rotated copy of the dataset and shuffle the result)
def augment_dataset(ds):
    skip = False
    if skip is False:
        # Normalize images
        print('Normalizing images...   0%', end='\r')
        # Calculate min an max for data and labels
        min_data = np.min(np.min(ds, axis=-1), axis=-1)
        max_data = np.max(np.max(ds, axis=-1), axis=-1)
        # Normalize all images to [0, 1]
        data_length = ds.shape[0]
        ds_ = ds.copy().astype(np.float64)
        for i in range(data_length):
            ds_[i] = (ds[i].astype(np.float64) - min_data[i]) / (max_data[i] - min_data[i])
            print(f'Normalizing images... {i/data_length*100:3.0f}%', end='\r')

        print(f'Normalizing images... {bcolors.OKGREEN}100%{bcolors.ENDC}')
    
    # Rotate images
    print('Augmenting dataset...', end='\r')
    # Create new dataset with rotated images and labels
    ds_ = np.concatenate((ds_, np.rot90(ds_, k=1, axes=(1,2)), np.rot90(ds_, k=2, axes=(1,2)), np.rot90(ds_, k=3, axes=(1,2))))
    print(f'Augmenting dataset... {bcolors.OKGREEN}Done{bcolors.ENDC}')
    # Shuffle the dataset
    print('Shuffling dataset...', end='\r')
    rng = np.random.default_rng()
    idx = rng.permutation(ds_.shape[0])
    ds_= ds_[idx]
    print(f'Shuffling dataset... {bcolors.OKGREEN}Done{bcolors.ENDC}')
    
    return ds_

In [43]:
dataset = load_sim_dataset('DNN4SIM_data/data/raw', channels='3D-SIM 640')

C:\Users\kay-1\Google Drive\DNN4SIM\env\lib\site-packages\nd2reader\raw_metadata.py:171: UserWarning: Z-levels details missing in metadata. Using Z-coordinates instead.
  warnings.warn("Z-levels details missing in metadata. Using Z-coordinates instead.")


File missing: DNN4SIM_data\data\raw\Image057\SIM_output.nd2


In [44]:
aug_dataset = augment_dataset(dataset)

Normalizing images... 100%
Augmenting dataset... Done
Shuffling dataset... Done


In [45]:
print('Data: ', aug_dataset.shape)

# Show a random collection of 5 images and their corresponding labels
r_0 = np.random.randint(0, aug_dataset.shape[0], 4)
img_list = []
for i in range(len(r_0)):
    img_list.append(aug_dataset[r_0[i]])

plt.close('all')
view = viewer(img_list, subplots=(2,2))

Data:  (476, 1024, 1024)


Button(description='Show Widgets', style=ButtonStyle())

# Save augmented dataset as Matlab file(s)

In [46]:
import scipy.io as sio
ch = '3D-SIM 640' # Channel
for i in range(4):
    with open(f'DNN4SIM_data/dataset_labels_{i}_{ch}.mat', 'wb') as f:
        sio.savemat(f, {'data' : aug_dataset[i*aug_dataset.shape[0]//4:(i+1)*aug_dataset.shape[0]//4]})

# Image Acquisition

In [56]:
%% Reading data
%fprintf('Reading data .............');
%im=double(loadtiff(gtpath)); im=im/max(im(:));
%fprintf(' done \n');
%% Image Noiseless Acquisition

ch = '3D-SIM 488'; % Channel

fprintf('Acquisition simulation ...');
img_count = 0;
for mat_nb = 0:3
    load(['DNN4SIM_data/dataset_labels_' num2str(mat_nb) '_' ch '.mat']);
    for img_idx = 1:size(data,1)
        im = squeeze(data(img_idx, :, :));
        % - LinOp Downsampling and integration over camera pixels
        SS=LinOpIdentity(sz);
        S=LinOpDownsample(sz(1:2),downFact);
        % htilde=padarray(ones(downFact),(sz(1:2)-downFact)/2,0,'both');
        % Htilde=LinOpConv(fftn(fftshift(htilde)));
        % - LinOpConv (PSF)
        OTF=Sfft(fftshift(fftshift(psf(:,:,end:-1:1),1),2),3);
        H=LinOpConv(OTF,1,[1 2]);
        % - Acquisition
        acqNoNoise=zeros([S.sizeout,size(patt,3)]);
        fprintf(' Pattern # ');
        for it=1:size(patt,3)
            fprintf([num2str(it),'..']);
            D=LinOpDiag(sz,patt(:,:,it));
        %     acqNoNoise(:,:,it)=S*Htilde*SS*H*D*im;
            acqNoNoise(:,:,it)=S*SS*H*D*im;
        end
        fprintf(' done \n');

        %% Add noise and Save
        for ii=1:length(photBud)
            % - Add Noise
            acq=acqNoNoise;
            if photBud>0
                tmp=sum(acqNoNoise,3);
                factor = photBud(ii)./mean(tmp(:)) ;
                acqNoNoise = acqNoNoise.* factor;
                im = im.*factor;
                acq = random('Poisson',acqNoNoise);
            end
            %     acq=poissrnd(round(acqNoNoise/sum(acqNoNoise(:))*photBud(ii)*prod(sz)));
            %     acqWF=poissrnd(round(acqWFNoNoise/sum(acqWFNoNoise(:))*photBud(ii)*prod(sz)));
            SNR=20*log10(norm(acqNoNoise(:))/norm(acq(:)-acqNoNoise(:)));
            disp(['SNR = ',num2str(SNR),' dB']);

            % - Save
            if sav
                expFolder_ = [expFolder '_' ch '_' num2str(img_count)];
                saveastiff(single(acqNoNoise),[expFolder_,'/AcqDataNoiseless.tif']);
                %saveastiff(single(log(1+abs(fftshift(fftshift(Sfft(acqNoNoise,3),1),2)))),[expFolder_,'/AcqDataNoiseless-FFT.tif']);
                saveastiff(single(acq),[expFolder_,'/AcqData.tif']);
                %saveastiff(single(log(1+abs(fftshift(fftshift(Sfft(acq,3),1),2)))),[expFolder_,'/AcqData-FFT.tif']);
                saveastiff(single(sum(acq,3)),[expFolder_,'/WFData.tif']);
                %saveastiff(single(log(1+abs(fftshift(fft2(sum(acq,3)))))),[expFolder_,'/WFData-FFT.tif']);
                saveastiff(single(sum(acqNoNoise,3)),[expFolder_,'/WFDataNoiseless.tif']);
                %saveastiff(single(log(1+abs(fftshift(fft2(sum(acqNoNoise,3)))))),[expFolder_,'/WFDataNoiseless-FFT.tif']);
            end
        end
        img_count = img_count + 1;
        break % for testing
    end
end

Acquisition simulation ... Pattern # 1..2..3..4..5..6..7..8..9.. done 
SNR = 17.7661 dB
 Pattern # 1..2..3..4..5..6..7..8..9.. done 
SNR = 19.9309 dB
 Pattern # 1..2..3..4..5..6..7..8..9.. done 
SNR = 22.015 dB
 Pattern # 1..2..3..4..5..6..7..8..9.. done 
SNR = 19.2014 dB
